In [ ]:
## import (and install if necessary) pandas
%pip install pandas
import pandas as pd

In [35]:
df = pd.read_csv("full_data.csv"
                 ,parse_dates=['TripDate','StartDateTime','EndDateTime','CustomerBirthDate']
                 )
## process cost monetary data
df['Cost'] = df['Cost'].str.replace(',', '')
df['Cost'] = df['Cost'].str.replace('$', '')
df['Cost'] = df['Cost'].str.replace('(', '')
df['Cost'] = df['Cost'].str.replace(')', '')
df['Cost'] = df['Cost'].astype(float)
print(df.info())
print(df.head(5))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1160755 entries, 0 to 1160754
Data columns (total 14 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   CustomerID         1160755 non-null  int64         
 1   CustomerSex        1160755 non-null  object        
 2   CustomerBirthDate  1160417 non-null  datetime64[ns]
 3   TripDate           1160755 non-null  datetime64[ns]
 4   VehicleID          1160755 non-null  int64         
 5   RunActualID        1160755 non-null  int64         
 6   StartDateTime      1160755 non-null  datetime64[ns]
 7   EndDateTime        1160755 non-null  datetime64[ns]
 8   PickupX            1110755 non-null  float64       
 9   PickupY            1110755 non-null  float64       
 10  DropoffX           1109028 non-null  float64       
 11  DropoffY           1109028 non-null  float64       
 12  Cost               1160538 non-null  float64       
 13  CollectedFareType  1147941 